# Lecture 2: Convolution and Linearity Testing

**Based on CS294-92: Analysis of Boolean Functions (Spring 2025)**  
**Instructor: Avishay Tal**  
**Scribe: Austin Pechan**

This notebook covers:
1. Expectation and Variance of Boolean Functions
2. Convolution in Fourier Domain
3. The BLR Linearity Test
4. Local Correction

---

## Key Theorems (Recap)

- **Fundamental Theorem**: $f(x) = \sum_{S \subseteq [n]} \hat{f}(S) \cdot \chi_S(x)$
- **Plancherel**: $\langle f, g \rangle = \sum_S \hat{f}(S) \hat{g}(S)$
- **Parseval**: $\sum_S \hat{f}(S)^2 = 1$ for Boolean-valued $f$

In [ ]:
import numpy as np
import boofun as bf
from boofun.analysis import SpectralAnalyzer
from boofun.analysis import PropertyTester
from boofun.analysis.fourier import parseval_verify, plancherel_inner_product, convolution

import warnings
warnings.filterwarnings('ignore')

## 2.1 Expectation and Variance

**Fact 2.6**: $\mathbf{E}[f(x)] = \hat{f}(\emptyset)$

The constant Fourier coefficient is the expectation!

**Fact 2.7**: $\mathbf{Var}[f] = \sum_{S \neq \emptyset} \hat{f}(S)^2$

For Boolean functions: $\mathbf{Var}[f] = 1 - \hat{f}(\emptyset)^2$

In [ ]:
# Demonstrate expectation = f̂(∅)
functions = {
    "AND₃": bf.AND(3),
    "OR₃": bf.OR(3),  
    "MAJORITY₃": bf.majority(3),
    "PARITY₃": bf.parity(3),
}

print("Expectation = f̂(∅) verification:")
print(f"{'Function':<12} | {'E[f] direct':<12} | {'f̂(∅)':<12} | Match")
print("-" * 55)

for name, f in functions.items():
    # Direct expectation in ±1
    tt = np.asarray(f.get_representation("truth_table"))
    pm_vals = 1 - 2 * tt  # Convert to ±1
    expectation = np.mean(pm_vals)
    
    # Fourier coefficient at ∅ - direct API: f.fourier()
    fourier = f.fourier()
    f_hat_empty = fourier[0]
    
    match = "✓" if abs(expectation - f_hat_empty) < 1e-10 else "✗"
    print(f"{name:<12} | {expectation:<12.4f} | {f_hat_empty:<12.4f} | {match}")

In [ ]:
# Demonstrate variance = Σ_{S≠∅} f̂(S)² = 1 - f̂(∅)²
print("\nVariance verification:")
print(f"{'Function':<12} | {'Var direct':<12} | {'1 - f̂(∅)²':<12} | {'Σ f̂(S)²':<12}")
print("-" * 65)

for name, f in functions.items():
    tt = np.asarray(f.get_representation("truth_table"))
    pm_vals = 1 - 2 * tt
    var_direct = np.var(pm_vals)
    
    # Direct API: f.fourier()
    fourier = f.fourier()
    
    f_hat_empty = fourier[0]
    var_formula = 1 - f_hat_empty**2
    
    # Sum of squares of non-empty coefficients  
    sum_sq = sum(c**2 for c in fourier[1:])
    
    print(f"{name:<12} | {var_direct:<12.4f} | {var_formula:<12.4f} | {sum_sq:<12.4f}")

## 2.2 Convolution

**Definition**: The convolution of $f$ and $g$ is:
$$f * g(x) = \mathbf{E}_{y}[f(y) \cdot g(x \cdot y)]$$

where $x \cdot y$ denotes coordinate-wise multiplication in $\{\pm 1\}^n$.

**Theorem 2.8 (Convolution Theorem)**: 
$$\widehat{f * g}(S) = \hat{f}(S) \cdot \hat{g}(S)$$

Convolution in the time domain = multiplication in the Fourier domain!

In [ ]:
# Demonstrate convolution theorem
f = bf.majority(3)
g = bf.parity(3)

# Get Fourier expansions - direct API: f.fourier()
f_fourier = f.fourier()
g_fourier = g.fourier()

# Compute convolution using the library
conv_result = convolution(f, g)
conv_fourier = conv_result.fourier()

# Expected: (f*g)^(S) = f^(S) * g^(S)
expected_fourier = f_fourier * g_fourier

print("Convolution Theorem Verification:")
print(f"{'S':<8} | {'f̂(S)':<10} | {'ĝ(S)':<10} | {'f̂·ĝ':<10} | {'(f*g)^':<10}")
print("-" * 60)

for i, (fc, gc, ec, cc) in enumerate(zip(f_fourier, g_fourier, expected_fourier, conv_fourier)):
    if abs(fc) > 0.01 or abs(gc) > 0.01 or abs(ec) > 0.01:
        print(f"{i:<8} | {fc:<10.4f} | {gc:<10.4f} | {ec:<10.4f} | {cc:<10.4f}")

## 2.3 BLR Linearity Test

The **BLR Test** (Blum-Luby-Rubinfeld, 1993) tests whether a function $F: \mathbb{F}_2^n \to \{\pm 1\}$ is **linear** (a character $\chi_S$).

**Algorithm**:
1. Pick $x, y \sim \mathbb{F}_2^n$ uniformly at random
2. Query $F(x)$, $F(y)$, and $F(x + y)$
3. **Accept** if $F(x) \cdot F(y) = F(x + y)$

**Key Insight**: For a linear function $\chi_S$:
$$\chi_S(x) \cdot \chi_S(y) = \chi_S(x + y)$$
Always! So linear functions pass with probability 1.

**Theorem 2.10**: $\Pr[\text{BLR accepts } F] = \sum_{S \subseteq [n]} \hat{F}(S)^3$

**Corollary**: If $F$ is $\varepsilon$-close to linear, BLR rejects with probability $\leq \varepsilon$.

In [ ]:
# Test linearity using BLR test from boofun
test_functions = {
    "PARITY₄ (linear!)": bf.parity(4),
    "x₁ (dictator, linear!)": bf.dictator(4, 0),
    "MAJORITY₅ (not linear)": bf.majority(5),
    "AND₄ (not linear)": bf.AND(4),
}

print("BLR Linearity Testing:")
print(f"{'Function':<25} | {'Is Linear?':<12} | {'BLR Result'}")
print("-" * 60)

for name, f in test_functions.items():
    tester = PropertyTester(f)
    # Run BLR test with many queries for high confidence
    blr_result = tester.blr_linearity_test(100)
    is_linear = f.is_linear()
    status = "✓ PASS" if blr_result else "✗ FAIL"
    print(f"{name:<25} | {str(is_linear):<12} | {status}")

In [ ]:
# Verify Theorem 2.10: Pr[accept] = Σ f̂(S)³
print("\nBLR Acceptance Probability = Σ f̂(S)³:")
print(f"{'Function':<25} | {'Σ f̂(S)³':<12} | {'Empirical'}")
print("-" * 60)

def blr_empirical_acceptance(f, trials=10000):
    """Run BLR test many times and compute acceptance rate."""
    n = f.n_vars
    accepts = 0
    
    for _ in range(trials):
        x = np.random.randint(0, 2, n)
        y = np.random.randint(0, 2, n)
        xy = (x + y) % 2  # XOR in F_2
        
        fx = 1 - 2 * f.evaluate(x)  # Convert to ±1
        fy = 1 - 2 * f.evaluate(y)
        fxy = 1 - 2 * f.evaluate(xy)
        
        if fx * fy == fxy:
            accepts += 1
    
    return accepts / trials

for name, f in test_functions.items():
    # Direct API: f.fourier()
    fourier = f.fourier()
    sum_cubed = sum(c**3 for c in fourier)
    empirical = blr_empirical_acceptance(f)
    print(f"{name:<25} | {sum_cubed:<12.4f} | {empirical:<12.4f}")

## 2.4 Local Correction

Once we know $F$ is $\varepsilon$-close to some linear function $\chi_S$, we can **locally correct** it!

**LocalCorrect(F, x)**:
1. Choose $y \sim \mathbb{F}_2^n$ uniformly
2. Query $F(y)$ and $F(x + y)$
3. Return $F(y) \cdot F(x + y)$

**Theorem 2.15**: For all $x$:
$$\Pr_y[\text{LocalCorrect}(F, x) = \chi_S(x)] \geq 1 - 2\varepsilon$$

This gives us access to $\chi_S$ even without knowing $S$!

In [ ]:
# Demonstrate local correction
def create_noisy_parity(n, noise_rate=0.1):
    """Create a parity function with some noise."""
    parity = bf.parity(n)
    tt = np.asarray(parity.get_representation("truth_table")).copy()
    
    # Flip some bits randomly
    flip_mask = np.random.random(len(tt)) < noise_rate
    tt = (tt + flip_mask.astype(int)) % 2
    return bf.create(tt.tolist())

def local_correct(f, x, repetitions=10):
    """Local correction with majority voting."""
    n = f.n_vars
    votes = []
    for _ in range(repetitions):
        y = np.random.randint(0, 2, n)
        xy = (x + y) % 2
        fy = 1 - 2 * f.evaluate(y)
        fxy = 1 - 2 * f.evaluate(xy)
        votes.append(fy * fxy)
    return 1 if sum(votes) > 0 else -1

# Test with a noisy parity function
n = 5
true_parity = bf.parity(n)
noisy_parity = create_noisy_parity(n, noise_rate=0.1)

# Measure distance to true parity
tt_true = np.asarray(true_parity.get_representation("truth_table"))
tt_noisy = np.asarray(noisy_parity.get_representation("truth_table"))
noise_level = np.mean(tt_true != tt_noisy)
print(f"Noisy parity: {noise_level:.1%} of bits flipped")

# Test local correction
correct = 0
for i in range(2**n):
    x = np.array([int(b) for b in format(i, f'0{n}b')])
    corrected = local_correct(noisy_parity, x)
    true_val = 1 - 2 * true_parity.evaluate(x)
    if corrected == true_val:
        correct += 1

print(f"Local correction accuracy: {correct / 2**n:.1%}")

## Summary

### Key Takeaways from Lecture 2:

1. **Expectation & Variance**: 
   - $\mathbf{E}[f] = \hat{f}(\emptyset)$ (constant coefficient)
   - $\mathbf{Var}[f] = 1 - \hat{f}(\emptyset)^2$ for Boolean functions

2. **Convolution Theorem**: $\widehat{f * g}(S) = \hat{f}(S) \cdot \hat{g}(S)$

3. **BLR Linearity Test**: 
   - Tests if $F$ is linear with only 3 queries
   - Acceptance probability = $\sum_S \hat{F}(S)^3$
   - If $\varepsilon$-close to linear, accepts with prob $\geq 1 - \varepsilon$

4. **Local Correction**: 
   - Can correct any input with high probability
   - Uses $F(y) \cdot F(x+y)$ to compute $\chi_S(x)$

### Using boofun (Direct API):

```python
from boofun.analysis import PropertyTester
from boofun.analysis.fourier import convolution

# Linearity test
tester = PropertyTester(f)
is_linear = tester.blr_linearity_test(num_queries=100)

# Convolution
h = convolution(f, g)

# Fourier analysis - direct method!
fourier = f.fourier()  # No SpectralAnalyzer needed
```